<a href="https://colab.research.google.com/github/justin900302/AI/blob/main/9_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##查看GPU

In [1]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


##安裝需要的BERT套件

In [2]:
!pip install transformers -U
!pip install nlp -U
!pip install torch -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 131.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##下載藥品療效資料檔

In [3]:
!wget https://github.com/shhuangmust/AI/raw/111-2/train.csv

--2023-05-25 15:17:30--  https://github.com/shhuangmust/AI/raw/111-2/train.csv
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/shhuangmust/AI/111-2/train.csv [following]
--2023-05-25 15:17:30--  https://raw.githubusercontent.com/shhuangmust/AI/111-2/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 388895 (380K) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>] 379.78K  --.-KB/s    in 0.04s   

2023-05-25 15:17:30 (10.1 MB/s) - ‘train.csv’ saved [388895/388895]



##載入套件

In [4]:
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
from nlp import load_dataset, Dataset
import nlp
import torch
import random
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

##讀取訓練資料集

In [5]:
df = pd.read_csv('train.csv')
dataset = Dataset.from_pandas(df)

In [6]:
df.sample(10)

,ID,Name,Description,label
713,713,顧衛能(美樂家),含草本舒緩精華，薑根/小茴香/洋甘菊/碳酸鈣等，能幫助消化，舒緩不舒服。每日三餐飯後各一錠，...,0
748,748,葡眾樂優乳酸菌顆粒(葡眾),本品嚴選優質益生菌種，為產學合作創新研發之新時代商品，因應國人應酬頻繁且飲食不均之現況，為外...,0
848,848,https://tw.buy.yahoo.com/gdsale/GNC健安喜-準媽咪必備-婦...,【婦寶樂食品錠】準媽媽專用綜合維他命。適合懷孕前、懷孕中和產後的營養補充。特別加強葉酸、鐵質...,0
398,398,白金葡萄籽300顆,「(網址1)紫爆空汙你吃對了嗎？亮白神器！自體發光！...(網址2)花青素主要功效為清除自由...,1
794,794,仕女如沛青春配方-NK,LifePak?Women仕女如沛?是專為18歲至熟齡前的女性設計，提供維生素、礦物質及植物...,0
886,886,https://tw.buy.yahoo.com/gdsale/GNC健安喜-美諾婷飲品經典...,低熱量、高纖維且營養均衡的替代飲品，每份熱量174-178大卡，碳水化合物30-31公克、優...,0
570,570,速養療、福寧補,「…記者陳O珍台北報導…癌症…化療副作用…罹癌後的種種病痛…提供癌症病患的營養補充，以作身體...,1
658,658,愛尚它?強鈣配方粉末,鈣是一種不可或缺的元素。愛尚它?強鈣配方粉末透過等滲壓溶液，提供適量的鈣、維生素D3、維生素...,0
805,805,康采R.N.A天山精華飲-avon,啟動生命源力延長黃金人生300毫克CellYoung?雪蓮組織培養物：每瓶添加300毫克「C...,0
527,527,益兒壯嬰幼兒口服電解質補充液,「運動前後感冒、發燒、腹瀉．．．」,1


##載入原始模型

In [7]:
model = BertForSequenceClassification.from_pretrained('hfl/chinese-bert-wwm-ext')

Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

##載入分詞模型

In [8]:
tokenizer = BertTokenizerFast.from_pretrained('hfl/chinese-bert-wwm-ext')

##設定執行參數

In [9]:
RANDOM_SEED = 5
MAX_LEN = 512
EPOCHS=5
BATCH_SIZE = 8

##把資料集分成訓練、測試、驗證

In [10]:

shuffled_ds = dataset.shuffle(RANDOM_SEED)
split_ds = shuffled_ds.train_test_split(test_size=0.2)
train_dataset = split_ds['train']
test_val_dataset = split_ds['test']
split_tv = test_val_dataset.train_test_split(test_size=0.5)
test_dataset = split_tv['train']
val_dataset = split_tv['test']

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

##取一筆出來瞧瞧(已經打亂)

In [11]:
train_dataset[2]

{'Description': '「實證案例-異位性皮膚炎…使用S1時間為2個月…使用S1時間為3個月」',
 'ID': 200,
 'Name': 'Nutrilon諾優貝',
 'label': 1}

In [12]:
tokenizer.tokenize("體重跟體脂都有在持續緩慢的下降中")

['體',
 '重',
 '跟',
 '體',
 '脂',
 '都',
 '有',
 '在',
 '持',
 '續',
 '緩',
 '慢',
 '的',
 '下',
 '降',
 '中']

##定義分詞器

In [13]:
def tokenize(batch):
  return tokenizer(batch['Description'], max_length=MAX_LEN, padding=True, truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

##定義評測準則，訓練器

In [15]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=50,
    weight_decay=0.01,
    # evaluate_during_training=True,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)
     

ImportError: ignored

##開始訓練

In [16]:
trainer.train()
trainer.evaluate()
     

NameError: ignored

##將模型存檔

In [ ]:
model.save_pretrained("bert_food_ad")

##用TENSORBOARD查看訓練結果

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir='./logs' # --host 192.168.1.113
#from tensorboard import notebook
#notebook.list()
#notebook.display(port=6006, height=1000)
     

##載入已訓練好的模型


In [ ]:
from transformers import BertConfig

test_config = BertConfig.from_json_file('./bert_food_ad/config.json')
test_model = BertForSequenceClassification.from_pretrained('./bert_food_ad/pytorch_model.bin', config=test_config)
model.eval()

In [ ]:
print(df.iloc[220]['label'], df.iloc[220]['Description'])
print(df.iloc[578]['label'], df.iloc[578]['Description'])
     

##在此輸入測試文字

In [ ]:
test_text = '增強免疫力...護眼明目...抗氧化...保肝...皮膚：...保護皮膚，減緩皮屑問題，降低敏感...毛髮……促進毛髮亮麗...消化系統」「強化免疫系統，有助調整過敏體質...清除自由基延緩老化...調整腸胃道機能...調理皮膚問題...諾麗果含有許多強大的抗氧化劑」「阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外」「它能維護身體細胞組織正常運作，阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外，使身體恢復正常'

MAX_LEN = 512

encoded_review = tokenizer.encode_plus(
    test_text,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=False,
    return_attention_mask=True,
    return_tensors='pt',
    truncation=True
)

In [ ]:
encoded_review

##進行測試

In [17]:
test_model = test_model.to('cuda')

input_ids = encoded_review['input_ids'].to('cuda')
attention_mask = encoded_review['attention_mask'].to('cuda')
output = test_model(input_ids, attention_mask)
result = torch.argmax(output[0][0])

classnames = ['Passed', 'Failed']

print(f'Review text: {test_text}')
print(f'Result  : {classnames[result]}')

NameError: ignored